In [3]:
!uv pip install pandas

Using Python 3.12.1 environment at: /workspaces/End2End_RAG/.venv
Resolved 6 packages in 824ms                                         
⠙ Preparing packages... (0/3)                                                   
⠙ Preparing packages... (0/3)-------------------     0 B/11.79 MiB           
⠙ Preparing packages... (0/3)------------------- 14.90 KiB/11.79 MiB         
⠙ Preparing packages... (0/3)------------------- 14.90 KiB/11.79 MiB         
pytz                 ------------------------------     0 B/497.29 KiB
⠙ Preparing packages... (0/3)------------------- 14.90 KiB/11.79 MiB         
pytz                 ------------------------------     0 B/497.29 KiB
⠙ Preparing packages... (0/3)------------------- 14.90 KiB/11.79 MiB         
pytz                 ------------------------------     0 B/497.29 KiB
⠙ Preparing packages... (0/3)------------------- 14.90 KiB/11.79 MiB         
tzdata               ------------------------------     0 B/339.69 KiB
pytz                 ----------

In [1]:
import pandas as pd

inputs = [
    "Why does Anne begin attending church even though she does not value the sermons?",
    "What inner conflict does Anne experience after receiving old Willy’s bequest?",
    "How does Anne learn the true challenges of working in the garden?",
]

outputs = [
    "Anne attends church not for the sermon, but for a sense of 'keeping step' with the community, as Miss Abby explains that people go to stay in rhythm with others rather than for the preaching.",

    "Anne feels deep self-reproach after Willy's bequest because the old woman had loved and depended on her long before Anne ever intended to care for her. She realizes she only stayed because of her own need, not Willy’s, and confronts her past failures toward those who depended on her.",

    "Anne discovers the real difficulty of gardening when her first attempts leave her tired, frustrated, and unsuccessful. She learns that working with nature requires patience and timing, and that the earth only becomes workable when it is ready—not when she wants it to be.",
]

qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
df = pd.DataFrame(qa_pairs)

# Write to csv
csv_path = "/workspaces/End2End_RAG/data/goldens.csv"
df.to_csv(csv_path, index=False)


In [2]:
from langsmith import Client

client = Client()
dataset_name = "AgenticAIReportGoldens"

# Store
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Input and expected output pairs for AgenticAIReport",
)
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

{'example_ids': ['1419eae8-06e5-4e53-97ac-6652dda62e2e',
  'dee59ebc-2745-4460-8b09-8ec78d016bff',
  '16a9b6b4-6427-4eba-a067-41dc15b72efd'],
 'count': 3}

In [4]:
import sys
sys.path.append("/workspaces/End2End_RAG")

from pathlib import Path
from multi_doc_chat.src.document_ingestion.document_ingestion import ChatIngestor
from multi_doc_chat.src.document_chat.retrieval import ConversationalRAG
import os

# Simple file adapter for local file paths
class LocalFileAdapter:
    """Adapter for local file paths to work with ChatIngestor."""
    def __init__(self, file_path: str):
        self.path = Path(file_path)
        self.name = self.path.name
    
    def getbuffer(self) -> bytes:
        return self.path.read_bytes()


def answer_ai_report_question(
    inputs: dict,
    data_path: str = "/workspaces/End2End_RAG/data/Way of All Earth.txt",
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    k: int = 5
) -> dict:
    """
    Answer questions about the AI Engineering Report using RAG.
    
    Args:
        inputs: Dictionary containing the question, e.g., {"question": "What is RAG?"}
        data_path: Path to the AI Engineering Report text file
        chunk_size: Size of text chunks for splitting
        chunk_overlap: Overlap between chunks
        k: Number of documents to retrieve
    
    Returns:
        Dictionary with the answer, e.g., {"answer": "RAG stands for..."}
    """
    try:
        # Extract question from inputs
        question = inputs.get("question", "")
        if not question:
            return {"answer": "No question provided"}
        
        # Check if file exists
        if not Path(data_path).exists():
            return {"answer": f"Data file not found: {data_path}"}
        
        # Create file adapter
        file_adapter = LocalFileAdapter(data_path)
        
        # Build index using ChatIngestor
        ingestor = ChatIngestor(
            temp_base="data",
            faiss_base="faiss_index",
            use_session_dirs=True
        )
        
        # Build retriever
        ingestor.built_retriver(
            uploaded_files=[file_adapter],
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            k=k
        )
        
        # Get session ID and index path
        session_id = ingestor.session_id
        index_path = f"faiss_index/{session_id}"
        
        # Create RAG instance and load retriever
        rag = ConversationalRAG(session_id=session_id)
        rag.load_retriever_from_faiss(
            index_path=index_path,
            k=k,
            index_name=os.getenv("FAISS_INDEX_NAME", "index")
        )
        
        # Get answer
        answer = rag.invoke(question, chat_history=[])
        
        return {"answer": answer}
        
    except Exception as e:
        return {"answer": f"Error: {str(e)}"}

In [5]:
test_input = {"question": "Why does Anne begin attending church even though she does not value the sermons?"}
result = answer_ai_report_question(test_input)
print("Question:", test_input["question"])
print("\nAnswer:", result["answer"])

{"timestamp": "2025-12-01T15:12:38.291843Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-12-01T15:12:38.293850Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-12-01T15:12:38.295290Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm_model"], "timestamp": "2025-12-01T15:12:38.302923Z", "level": "info", "event": "YAML config loaded"}
{"session_id": "session_20251201_151238_26d3462d", "temp_dir": "data/session_20251201_151238_26d3462d", "faiss_dir": "faiss_index/session_20251201_151238_26d3462d", "sessionized": true, "timestamp": "2025-12-01T15:12:38.310186Z", "level": "info", "event": "ChatIngestor initialized"}
{"uploaded": "Way of All Earth.txt", "saved_as": "data/session_20251201_151238_26d3462d/ad6ff42f.txt", "timestamp": "2025-12-01T15:12:38.325222Z", "level": "info", "event": "File saved for ingestion"}


Question: Why does Anne begin attending church even though she does not value the sermons?

Answer: I don't know. The context states that Miss Abby believes people don't go to church to hear the sermon, and that she herself now goes regularly. However, it does not explicitly state Anne's reasons for attending.


In [6]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

In [7]:
# Example: Test with all golden questions
print("Testing all questions from the dataset:\n")
for i, q in enumerate(inputs, 1):
    test_input = {"question": q}
    result = answer_ai_report_question(test_input)
    print(f"Q{i}: {q}")
    print(f"A{i}: {result['answer']}\n")
    print("-" * 80 + "\n")

{"timestamp": "2025-12-01T15:23:33.444157Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-12-01T15:23:33.445117Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-12-01T15:23:33.448408Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm_model"], "timestamp": "2025-12-01T15:23:33.457002Z", "level": "info", "event": "YAML config loaded"}
{"session_id": "session_20251201_152333_5de968de", "temp_dir": "data/session_20251201_152333_5de968de", "faiss_dir": "faiss_index/session_20251201_152333_5de968de", "sessionized": true, "timestamp": "2025-12-01T15:23:33.459917Z", "level": "info", "event": "ChatIngestor initialized"}
{"uploaded": "Way of All Earth.txt", "saved_as": "data/session_20251201_152333_5de968de/1f4d0614.txt", "timestamp": "2025-12-01T15:23:33.467053Z", "level": "info", "event": "File saved for ingestion"}


Testing all questions from the dataset:



{"added": 1, "index": "faiss_index/session_20251201_152333_5de968de", "timestamp": "2025-12-01T15:28:18.407568Z", "level": "info", "event": "FAISS index updated"}
{"k": 5, "fetch_k": 20, "lambda_mult": 0.5, "timestamp": "2025-12-01T15:28:18.408381Z", "level": "info", "event": "Using MMR search"}
{"timestamp": "2025-12-01T15:28:18.410787Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-12-01T15:28:18.411284Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-12-01T15:28:18.412189Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm_model"], "timestamp": "2025-12-01T15:28:18.413748Z", "level": "info", "event": "YAML config loaded"}
{"provider": "google", "model": "gemini-2.5-flash", "timestamp": "2025-12-01T15:28:18.414604Z", "level": "info", "event": "Loading LLM"}
{"session_id": "session_20251201_152333_5de968de", 

Q1: Why does Anne begin attending church even though she does not value the sermons?
A1: I don't know. The context states that Miss Abby believes people don't go to church to hear the sermon, and that she herself now goes regularly. However, it does not explicitly state Anne's reasons for attending.

--------------------------------------------------------------------------------



{"added": 1, "index": "faiss_index/session_20251201_152826_2c08ec2e", "timestamp": "2025-12-01T15:33:10.523475Z", "level": "info", "event": "FAISS index updated"}
{"k": 5, "fetch_k": 20, "lambda_mult": 0.5, "timestamp": "2025-12-01T15:33:10.524474Z", "level": "info", "event": "Using MMR search"}
{"timestamp": "2025-12-01T15:33:10.527794Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-12-01T15:33:10.528594Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-12-01T15:33:10.529592Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm_model"], "timestamp": "2025-12-01T15:33:10.531934Z", "level": "info", "event": "YAML config loaded"}
{"provider": "google", "model": "gemini-2.5-flash", "timestamp": "2025-12-01T15:33:10.532856Z", "level": "info", "event": "Loading LLM"}
{"session_id": "session_20251201_152826_2c08ec2e", 

Q2: What inner conflict does Anne experience after receiving old Willy’s bequest?
A2: After receiving old Willy's bequest, Anne experiences a new feeling of dull self-reproach and a consciousness of failure. She realizes that Willy had loved her and perhaps dreamed of her care in her old age, a role Anne did not fulfill, despite the "kindly falsehood" she tells neighbors about coming to take care of her. This feeling is unmixed with anger, unlike her past conflicts.

--------------------------------------------------------------------------------



{"added": 1, "index": "faiss_index/session_20251201_153317_bde36f2d", "timestamp": "2025-12-01T15:37:59.309851Z", "level": "info", "event": "FAISS index updated"}
{"k": 5, "fetch_k": 20, "lambda_mult": 0.5, "timestamp": "2025-12-01T15:37:59.310944Z", "level": "info", "event": "Using MMR search"}
{"timestamp": "2025-12-01T15:37:59.313596Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-12-01T15:37:59.314182Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-12-01T15:37:59.314837Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm_model"], "timestamp": "2025-12-01T15:37:59.316820Z", "level": "info", "event": "YAML config loaded"}
{"provider": "google", "model": "gemini-2.5-flash", "timestamp": "2025-12-01T15:37:59.317645Z", "level": "info", "event": "Loading LLM"}
{"session_id": "session_20251201_153317_bde36f2d", 

Q3: How does Anne learn the true challenges of working in the garden?
A3: Anne learns the true challenges of working in the garden when she attempts tasks like scraping out rows and making the dirt fine, which leaves her shoulders aching. She realizes that what she thought was simple is intricate, leading to feelings of humility and helplessness. She also discovers that seemingly trivial details, like how corn is planted, make a significant difference, and that tasks like weeding beets are laborious and back-breaking.

--------------------------------------------------------------------------------



In [9]:
!uv pip install -U langsmith

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.12.1 environment at: /workspaces/End2End_RAG/.venv
Resolved 19 packages in 2.02s                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)-------------------     0 B/110.71 KiB          
⠙ Preparing packages... (0/1)------------------- 14.85 KiB/110.71 KiB        
⠙ Preparing packages... (0/1)------------------- 30.85 KiB/110.71 KiB        
⠙ Preparing packages... (0/1)------------------- 42.29 KiB/110.71 KiB        
⠙ Preparing packages... (0/1)---------------- 53.27 KiB/110.71 KiB        
⠙ Preparing packages... (0/1)--------------- 58.75 KiB/110.71 KiB        
⠙ Preparing packages... (0/1)30m------------ 69.73 KiB/110.71 KiB        
⠙ Preparing packages... (0/1)---------- 78.85 KiB/110.71 KiB        
⠙ Preparing packages... (0/1)---------- 94.85 KiB/110.71 KiB        
⠙ Preparing packages... (0/1)---------- 110.71 KiB/110.71 KiB       
Prepared 1 package in 159ms              

In [11]:
!uv pip install langchain-experimental

Using Python 3.12.1 environment at: /workspaces/End2End_RAG/.venv
⠙ Resolving dependencies...                                                     

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Resolved 46 packages in 2.09s                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)---------------------     0 B/204.70 KiB        
⠙ Preparing packages... (0/1)--------------------- 14.87 KiB/204.70 KiB      
⠙ Preparing packages... (0/1)--------------------- 30.87 KiB/204.70 KiB      
⠙ Preparing packages... (0/1)--------------------- 46.87 KiB/204.70 KiB      
⠙ Preparing packages... (0/1)--------------------- 62.87 KiB/204.70 KiB      
⠙ Preparing packages... (0/1)2m------------------- 78.87 KiB/204.70 KiB      
⠙ Preparing packages... (0/1)----------------- 94.87 KiB/204.70 KiB      
⠙ Preparing packages... (0/1)30m-------------- 110.87 KiB/204.70 KiB     
⠙ Preparing packages... (0/1)------------ 126.87 KiB/204.70 KiB     
⠙ Preparing packages... (0/1)------------ 142.87 KiB/204.70 KiB     
⠙ Preparing packages... (0/1)------------ 158.87 KiB/204.70 KiB     
⠙ Preparing packages... (0

In [20]:
from langsmith.evaluation import evaluate

# 1. Choose an evaluator – "cot_qa" works here
qa_evaluator = ["cot_qa"]

dataset_name = "AgenticAIReportGoldens"

# 2. Run evaluation using your RAG function
experiment_results = evaluate(
    answer_ai_report_question,     # Your function to test
    data=dataset_name,       # List of evaluator names
    experiment_prefix="test-agenticAIReport-qa-rag",
    metadata={
        "variant": "RAG with FAISS and AI Engineering Report",
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "k": 5,
    },
)

print(experiment_results)


View the evaluation results for experiment: 'test-agenticAIReport-qa-rag-cb839cb2' at:
https://smith.langchain.com/o/a055ffe7-e26c-47b9-aceb-78787385e028/datasets/cb723103-018f-44cb-8504-fc1a432cd953/compare?selectedSessions=37d7415e-5ef3-40a6-be43-6e2322ebe42f




0it [00:00, ?it/s]{"timestamp": "2025-12-01T16:41:48.632037Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-12-01T16:41:48.632778Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-12-01T16:41:48.633722Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm_model"], "timestamp": "2025-12-01T16:41:48.635312Z", "level": "info", "event": "YAML config loaded"}
{"session_id": "session_20251201_164148_8c4c4890", "temp_dir": "data/session_20251201_164148_8c4c4890", "faiss_dir": "faiss_index/session_20251201_164148_8c4c4890", "sessionized": true, "timestamp": "2025-12-01T16:41:48.636491Z", "level": "info", "event": "ChatIngestor initialized"}
{"uploaded": "Way of All Earth.txt", "saved_as": "data/session_20251201_164148_8c4c4890/516998b3.txt", "timestamp": "2025-12-01T16:41:48.638108Z", "level": "info", "event": "File save

KeyboardInterrupt: 

In [21]:
from langsmith.schemas import Run, Example
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

def correctness_evaluator(run: Run, example: Example) -> dict:
    """
    Custom LLM-as-a-Judge evaluator for correctness.
    
    Correctness means how well the actual model output matches the reference output 
    in terms of factual accuracy, coverage, and meaning.
    
    Args:
        run: The Run object containing the actual outputs
        example: The Example object containing the expected outputs
    
    Returns:
        dict with 'score' (1 for correct, 0 for incorrect) and 'reasoning'
    """
    # Extract actual and expected outputs
    actual_output = run.outputs.get("answer", "")
    expected_output = example.outputs.get("answer", "")
    input_question = example.inputs.get("question", "")
    
    # Define the evaluation prompt
    eval_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an evaluator whose job is to judge correctness.

Correctness means how well the actual model output matches the reference output in terms of factual accuracy, coverage, and meaning.

- If the actual output matches the reference output semantically (even if wording differs), it should be marked correct.
- If the output misses key facts, introduces contradictions, or is factually incorrect, it should be marked incorrect.

Do not penalize for stylistic or formatting differences unless they change meaning."""),
        ("human", """<example>
<input>
{input}
</input>

<output>
Expected Output: {expected_output}

Actual Output: {actual_output}
</output>
</example>

Please grade the following agent run given the input, expected output, and actual output.
Focus only on correctness (semantic and factual alignment).

Respond with:
1. A brief reasoning (1-2 sentences)
2. A final verdict: either "CORRECT" or "INCORRECT"

Format your response as:
Reasoning: [your reasoning]
Verdict: [CORRECT or INCORRECT]""")
    ])
    
    # Initialize LLM (using Gemini as shown in your config)
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-pro",
        temperature=0
    )
    
    # Create chain and invoke
    chain = eval_prompt | llm
    
    try:
        response = chain.invoke({
            "input": input_question,
            "expected_output": expected_output,
            "actual_output": actual_output
        })
        
        response_text = response.content
        
        # Parse the response
        reasoning = ""
        verdict = ""
        
        for line in response_text.split('\n'):
            if line.startswith("Reasoning:"):
                reasoning = line.replace("Reasoning:", "").strip()
            elif line.startswith("Verdict:"):
                verdict = line.replace("Verdict:", "").strip()
        
        # Convert verdict to score (1 for correct, 0 for incorrect)
        score = 1 if "CORRECT" in verdict.upper() else 0
        
        return {
            "key": "correctness",
            "score": score,
            "reasoning": reasoning,
            "comment": f"Verdict: {verdict}"
        }
        
    except Exception as e:
        return {
            "key": "correctness",
            "score": 0,
            "reasoning": f"Error during evaluation: {str(e)}"
        }

In [22]:
# Run evaluation with the custom correctness evaluator
from langsmith.evaluation import evaluate

# Define evaluators - using custom correctness evaluator
evaluators = [correctness_evaluator]

dataset_name = "AgenticAIReportGoldens"

# Run evaluation
experiment_results = evaluate(
    answer_ai_report_question,
    data=dataset_name,
    evaluators=evaluators,
    experiment_prefix="agenticAIReport-correctness-eval",
    description="Evaluating RAG system with custom correctness evaluator (LLM-as-a-Judge)",
    metadata={
        "variant": "RAG with FAISS and AI Engineering Report",
        "evaluator": "custom_correctness_llm_judge",
        "model": "gemini-2.5-pro",
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "k": 5,
    },
)

print("\nEvaluation completed! Check the LangSmith UI for detailed results.")

View the evaluation results for experiment: 'agenticAIReport-correctness-eval-93ce3a36' at:
https://smith.langchain.com/o/a055ffe7-e26c-47b9-aceb-78787385e028/datasets/cb723103-018f-44cb-8504-fc1a432cd953/compare?selectedSessions=71002bd2-cefa-47ff-b160-055a152c4ebc




0it [00:00, ?it/s]{"timestamp": "2025-12-01T16:56:02.448647Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-12-01T16:56:02.450114Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-12-01T16:56:02.451467Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm_model"], "timestamp": "2025-12-01T16:56:02.453249Z", "level": "info", "event": "YAML config loaded"}
{"session_id": "session_20251201_165602_48181686", "temp_dir": "data/session_20251201_165602_48181686", "faiss_dir": "faiss_index/session_20251201_165602_48181686", "sessionized": true, "timestamp": "2025-12-01T16:56:02.454125Z", "level": "info", "event": "ChatIngestor initialized"}
{"uploaded": "Way of All Earth.txt", "saved_as": "data/session_20251201_165602_48181686/e3f81822.txt", "timestamp": "2025-12-01T16:56:02.455596Z", "level": "info", "event": "File save


Evaluation completed! Check the LangSmith UI for detailed results.


In [23]:
experiment_results

,inputs.question,outputs.answer,error,reference.answer,feedback.correctness,execution_time,example_id,id
0,Why does Anne begin attending church even thou...,I don't know. The context indicates that Anne ...,None,"Anne attends church not for the sermon, but fo...",1,298.775518,1419eae8-06e5-4e53-97ac-6652dda62e2e,019adad7-ba8e-7740-833c-cbd45737d96c
1,How does Anne learn the true challenges of wor...,Anne learns the true challenges of gardening w...,None,Anne discovers the real difficulty of gardenin...,1,290.020175,16a9b6b4-6427-4eba-a067-41dc15b72efd,019adadc-6915-752f-9bed-1c0c7f222748
2,What inner conflict does Anne experience after...,"After receiving old Willy's bequest, Anne expe...",None,Anne feels deep self-reproach after Willy's be...,1,290.523999,dee59ebc-2745-4460-8b09-8ec78d016bff,019adae0-f8ec-7149-acf6-b9f8f7fe9aeb
